# Location updates  
This notebook contains the process of updating from old (staging) locations to new (prod) locations on the data for the new widgets.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import requests
import os

## Locations

In [2]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [3]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


## Mangroves in protected areas

In [4]:
file_name = "Mangrove in WDPA July 2022.xlsx"
data_sheet = 'July 2022'
bucket_url = "https://storage.googleapis.com/mangrove_atlas/widget_data/"

data_url = 'https://storage.googleapis.com/mangrove_atlas/widget_data/Mangrove%20in%20WDPA%20July%202022.xlsx'

protected_df = pd.read_excel(data_url, sheet_name=data_sheet)
protected_df.columns = protected_df.columns.str.lower().str.replace(' ', '_')
protected_df.head()

,iso_ter,total_mangrove_area(ha),mangrove_area_protected(ha),percent_protected
0,ABW,45.938704,1.0,0.021768
1,AGO,28356.673109,146.0,0.005149
2,AIA,3.700790,0.0,0.000000
3,ARE,7444.860192,1323.0,0.177706
4,ASM,32.050071,0.0,0.000000


In [5]:
protected_final = protected_df.merge(locations[['iso', 'location_idn']], left_on='iso_ter', right_on='iso', how='left')
protected_final.head()

,iso_ter,total_mangrove_area(ha),mangrove_area_protected(ha),percent_protected,iso,location_idn
0,ABW,45.938704,1.0,0.021768,ABW,5d1bd36b-f200-593d-972a-bf69abdd912f
1,AGO,28356.673109,146.0,0.005149,AGO,27ceab8c-946e-5286-a06f-8bd98ec81f77
2,AIA,3.700790,0.0,0.000000,AIA,1ce4c2e5-8456-5db8-8e34-8bfe86083790
3,ARE,7444.860192,1323.0,0.177706,ARE,7ec6ba5a-73a9-5911-8f47-107a5ac4e750
4,ASM,32.050071,0.0,0.000000,ASM,404d005a-797d-5509-91eb-e17ed1069ed6


In [6]:
protected_final.drop(['iso', 'iso_ter', 'percent_protected'], axis=1, inplace=True)
protected_final.rename(columns={'location_idn': 'location_id',
                                'mangrove_area_protected(ha)':'protected_area',
                                'total_mangrove_area(ha)':'total_area'}, inplace=True)
protected_final['year'] = 2020
protected_final = protected_final[protected_final.location_id.notnull()]
protected_final.head()

,total_area,protected_area,location_id,year
0,45.938704,1.0,5d1bd36b-f200-593d-972a-bf69abdd912f,2020
1,28356.673109,146.0,27ceab8c-946e-5286-a06f-8bd98ec81f77,2020
2,3.700790,0.0,1ce4c2e5-8456-5db8-8e34-8bfe86083790,2020
3,7444.860192,1323.0,7ec6ba5a-73a9-5911-8f47-107a5ac4e750,2020
4,32.050071,0.0,404d005a-797d-5509-91eb-e17ed1069ed6,2020


In [7]:
protected_final.to_csv('../../../../data/UPDATED_protected_areas.csv', index=False)

In [8]:
def uploadDataApi(filePath: str, endpoint: str) -> str:
    """
    upload the data to the api
    Args:
        filePath (str): path to the file to upload
        endpoint (str): endpoint to upload to

    Returns:
        str: url of the uploaded file
    """
    files = {'file': open(filePath, 'rb')}
    r = requests.post(endpoint, files=files)
    return r.status_code

In [9]:
STAGING_API = 'https://mangrove-atlas-api-staging.herokuapp.com'
output_file = '../../../../data/UPDATED_protected_areas.csv'
uploadDataApi(output_file, f'{STAGING_API}/api/v2/widgets/protected-areas/import')

201